In [1]:
using SparseArrays
using LinearAlgebra
using Plots
using Statistics
using SuiteSparse

In [ ]:
function topggp(nelx,nely,volfrac = 0.5,settings = 'GGP' ,maxiteration = 25)
    starting_guess = 'Crosses'
    E=1
    nu=0.3
    E0 = 1;
    Emin = 1e-6
    A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12]
    A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6]
    B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4]
    B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2]
    KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11])

    nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx); #number of the nodes in columns
    edofVec = reshape(2*nodenrs[1:end-1,1:end-1].+1,nelx*nely,1) ;#1st dof of each element (x top left)
    edofMat = zeros(nelx*nely, 8); #every line i contains the 8 dof of the ith element
    noeudsvoisins = [0 1 2*nely.+[2 3 0 1] -2 -1];
    for i = 1:8
        for j = 1:nelx*nely
            edofMat[j,i]= edofVec[j]+ noeudsvoisins[i] ;
        end
    end
    iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);# line to build  K
    jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);# columns
    
        # Define the nodal coordinates
    x=0
    Yy=zeros((nelx+1)*(nely+1))
    Xx=zeros((nelx+1)*(nely+1))
    for i = 1:(nelx+2)
        for j = 1:(nely+2)
            Yy[x]= j
            Xx[x]= i
            x+=1
    Yy=nely+1-Yy
    Xx=Xx-1
    
        # Compute the centroid coordinates
    xc = zeros(nelx * nely)
    yc = zeros(nelx * nely)
    for xi = 1:nelx
        xc[xi*nely:(xi+1)*nely]=xi+0.5
    for yi = 1:nely
        yc[yi:nely:(nelx*nely)]=yi+0.5
    yc = nely - yc
                    
    centroid_coordinate=[ [xc[i],yc[i]] for i in 1:len(xc) ]
    a=-R #R défini dans les if du debut-> rajouter apres
    b=R
    [gpc,wc]=lgwt(Ngp,a,b) #implémentée en bas
    [gpcx,gpcy]=np.meshgrid(gpc,gpc)# à suivre...
    gpcx=gpcx
    gpcy=gpcy

In [211]:
function lgwt(N,a,b)
    # lgwt.m This script is for computing definite integrals using Legendre-Gauss Quadrature.Computes the Legendre - Gauss nodes and weights on an interval [a, b] with truncation order N
    # Suppose you have a continuous function f(x) which is defined on[a, b]  which you can evaluate at any x in [a, b].Simply evaluate itat all of the values contained in the x vector to obtain a vector f.Then compute the definite integral using sum(f. * w);
    # Written by Greg von Winckel - 02 / 25 / 2004
    N = N - 1
    N1 = N + 1
    N2 = N + 2
    xu = range(-1,stop =  1, length =N1)
    xu = xu'

    # Initial guess
    y = cos.((2 * (0:N)'.+1)*pi/(2*N+2)).+(0.27/N1)*sin.(pi*xu*N/N2)
    # Legendre-Gauss Vandermonde Matrix
    L=zeros(N1, N2) # vu l'algo on pourrait avoir juste 2 variables vectorielles lk-1 et lk-2 pour moins de stockage

    # Compute the zeros of the N + 1 Legendre Polynomial using the recursion relation and the Newton - Raphson method
    y0 = 2
    eps = 2.2204e-16
    Lp = 0
    # Iterate until new points are uniformly within epsilon of old points
    while maximum(abs.(y .- y0)) > eps
        L[:, 1].=1
        L[:, 2]=y
        for k = 3:N2
            L[:, k]=((2*(k-1) -1 ) * y[:] .* L[:, k-1] - (k-2) * L[:, k-2] )/ (k-1) ; 
        end
        Lp = N2 * (L[:, N1] - y[:] .* L[:, N2])./ (1 .- y[:].^2);
        y0 = y
        y = y0[:] .- L[:, N2]./ Lp[:];
    end
    # Linear map from [-1, 1] to[a, b]
    x = (a * (1 .- y) + b * (1 .+ y)) / 2
    # Compute the weights
    w = (b - a) / ((1 .- y[:].^2) .* Lp[:].^2) * (N2 / N1)^2
    return x, w , L
end

lgwt (generic function with 1 method)

In [212]:
x,w,l = lgwt(3,6,7)
x

3-element Array{Float64,1}:
 6.887298334620742
 6.5
 6.112701665379259